In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import shutil

# 检查并安装必要的库
import importlib
import subprocess

required_libraries = ['paramiko', 'scp']
for lib in required_libraries:
    try:
        importlib.import_module(lib)
        print(f"{lib} 已经安装。")
    except ImportError:
        print(f"{lib} 未安装，正在安装...")
        subprocess.check_call(['pip', 'install', lib])
        print(f"{lib} 安装完成。")
'''
# 导入库
import paramiko
from scp import SCPClient


# 服务器信息
hostname = 'XXXXX'
port = 22
username = 'root'
password = 'XXXXXXX'  # 替换为你的服务器密码

# 远程文件路径
remote_path = '/root/projects/apa_atingLab2019'
'''
# 本地保存路径（Colab 环境）
local_path = '/content/apa_atingLab2019'
'''
# 创建 SSH 客户端
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, port, username, password)

# 使用 SCP 复制文件
with SCPClient(ssh.get_transport()) as scp:
    scp.get(remote_path, local_path, recursive=True)  # recursive=True 用于复制文件夹

print(f"文件已从服务器复制到 Colab: {local_path}")
'''
# Google Drive 目标路径
drive_path = '/content/drive/My Drive/apa_atingLab2019'
'''
# 将文件从 Colab 复制到 Google Drive
shutil.copytree(local_path, drive_path)

print(f"文件已保存到 Google Drive: {drive_path}")
'''
import os
if os.path.exists(local_path):
  shutil.rmtree(local_path)
shutil.copytree(drive_path, local_path)
print(f"文件已从 Google Drive: 复制回来")

In [ ]:
# 安装 Miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# 将 Conda 添加到 PATH
import sys
sys.path.append('/usr/local/bin')

# 验证安装
!conda --version
# 创建 Conda 环境（例如命名为 bioinfo
!conda create -n bioinfo  -y

# 激活环境
import os
os.environ['PATH'] = '/usr/local/envs/bioinfo/bin:' + os.environ['PATH']
!conda activate bioinfo

# 进入代码目录
%cd /content/apa_atingLab2019

# 使用 pip 安装包
!pip install -r python_requirements.txt

# 使用conda 安装包
programs = [
    "mosdepth", "sra-tools", "star", "xz", "wget",
    "configparser", "numpy", "pandas", "pysam", "argparse",
    "pyfaidx", "Bio", "deeptools", "regex"
]

for prog in programs:
    !conda install -c bioconda {prog} -y


with open('python_requirements.txt', 'r') as f:
    required_packages = f.readlines()

# 清理空白字符和注释
required_packages = [line.strip() for line in required_packages if line.strip() and not line.startswith('#')]
print("Required packages:", required_packages)
missing_packages = []
for package in required_packages:
    try:
        # 提取包名和版本
        if '==' in package:
            package_name, required_version = package.split('==')
        else:
            package_name = package
            required_version = None

        # 检查是否安装
        !pip show {package_name} > /dev/null 2>&1
        installed_version = !pip show {package_name} | grep Version | awk '{{print $2}}'
        installed_version = installed_version[0] if installed_version else None

        if required_version and installed_version != required_version:
            print(f"WARNING: {package_name} is installed (version {installed_version}), but version {required_version} is required.")
        elif not installed_version:
            missing_packages.append(package_name)
    except Exception as e:
        missing_packages.append(package_name)

if missing_packages:
    print("Missing packages:", missing_packages)
else:
    print("All required packages are installed.")


# 安装 bowtie2,samtools,Java（picard 依赖）
!apt-get update
!apt-get install -y bowtie2
!apt-get install -y samtools
!apt-get install -y openjdk-11-jre


%cd /content/

# 下载 picard
!wget https://github.com/broadinstitute/picard/releases/download/2.27.5/picard.jar

# 下载 UCSC 工具集
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/wigToBigWig

# 赋予执行权限
!chmod +x twoBitToFa
!chmod +x wigToBigWig
'''
# 验证安装
!bowtie2 --version
!samtools --version
!java -jar picard.jar -h
!./twoBitToFa
!./wigToBigWig
'''

%cd /content/apa_atingLab2019

# 安装 R及R的扩展包
!conda install -c bioconda r-base=3.4.3 -y
r_packages = [
    "goldmine", "argparse", "BSgenome.Hsapiens.UCSC.hg19", "corrplot",
    "data.table", "DEXSeq", "e1071", "edgeR", "ggbio", "ggplot2", "gridExtra",
    "IlluminaHumanMethylation450kanno.ilmn12.hg19", "matrixStats", "openxlsx",
    "parallel", "reshape", "RPMM", "Rsamtools", "Rsubread", "stringr", "ggseqlogo",
    "JASPAR2018", "TFBSTools", "UpSetR", "VennDiagram", "wateRmelon", "devtools"
]

for pkg in r_packages:
    !Rscript -e "install.packages('{pkg}', repos='https://cloud.r-project.org/')"

# 安装 devtools 包
!Rscript -e "install.packages('devtools', repos='https://cloud.r-project.org/')"

# 克隆 Handy仓库并安装
%cd /content
!git clone https://github.com/jeffbhasin/handy.git
%cd /content/handy
!Rscript -e "library(devtools); install_github('jeffbhasin/handy')"

# 进入目标目录
%cd /content/apa_atingLab2019

# 设置环境变量
import os
os.environ['R_UTIL_APA'] = '/content/apa_atingLab2019/resource/libs'
os.environ['PADD_GIT'] = '/content/apa_atingLab2019/01_polyAseq/paddle-git'

# 将环境变量写入 ~/.bashrc（可选）
with open('/root/.bashrc', 'a') as f:
    f.write(f'\nexport R_UTIL_APA={os.environ["R_UTIL_APA"]}')
    f.write(f'\nexport PADD_GIT={os.environ["PADD_GIT"]}')

%cd /content/apa_atingLab2019/01_polyAseq
!ls


In [ ]:
!bash ./01_resource_prep.sh
#!bash ./02_get_fasterq.sh

In [ ]:
'''
programs = [
    "mosdepth", "sra-tools", "star", "xz", "wget",
    "configparser", "numpy", "pandas", "pysam", "argparse",
    "pyfaidx", "Bio", "deeptools", "regex"
]

for prog in programs:
    !conda install -c bioconda {prog} -y
    '''

In [ ]:
#!bash ./03_polyaseq.sh

In [ ]:

import os, sys, argparse
import inspect
from lib.util.lib_utils import run_syscmd_wrapper, get_abspath_or_error, ensure_dir, check_if_file_valid, msgout
from lib.util.lib_par import par_shell_runs, find_error_index_from_retcodes
from lib.file_configs import ProgramConfig

PIPELINE_NAME = 'polyAseq'
VERSION = '0.1'
author_email = 'hongc2@ccf.org'

def split_barcodes_core(fastq_dir,
												prog_path,
												out_dir,
												log_prefix,
												barcode_fn,
												max_mismatch=2,
												ncpu = 2,
												debug = False):
	'''
	Note: this def can be generalized to dispatch multiple commands
	:param fastq_dir:
	:param prog_path:
	:param exp_dir:
	:param max_mismatch:
	:param ncpu:
	:return:
	'''

	fns = os.listdir(fastq_dir)
	arg_sets = []

	for fn in fns:
		msgout("notice","checking if [%s] is FASTQ file ..."%fn)
		if fn.endswith('.fastq.gz') or fn.endswith('.fastq'):

			cmd = prog_path
			cmd += " --outdir %s" % out_dir
			cmd += " --mm %d" % max_mismatch
			cmd += " --barcode_fn %s" % barcode_fn
			cmd += " --inpath %s"%os.path.join(fastq_dir,fn)
			stdout_fn = "%s_%s.out" % (log_prefix,fn)
			stderr_fn = "%s_%s.err" % (log_prefix,fn)
			msgout("notice",cmd)
			arg_sets.append((cmd,stdout_fn,stderr_fn,False,))

	if not debug: #debug
		retcodes = par_shell_runs(arg_sets,ncpu=ncpu,job_name=inspect.stack()[0][3])
		error_idx = find_error_index_from_retcodes(retcodes)
		if error_idx:
			for j in error_idx:
				print(('an error in the command [%s]' % arg_sets[j][0]))
			raise RuntimeError('there are some errors in split_barcdes_core()')

	return arg_sets

def bam_index(mod_bam_dir,debug=False):

	bams = os.listdir(mod_bam_dir)
	for bam in bams:
		if bam.endswith('.bam'):
			bai = os.path.join(mod_bam_dir,bam+'.bai')
			cmd = "samtools index %s"%(os.path.join(mod_bam_dir,bam))
			run_syscmd_wrapper(cmd,debug=debug)

class mStep:
	def __init__(self,name1,cnt1):
		self.name = name1
		self.cnt = cnt1
		self.substeps = []
		self.out_dir = None

	def get_out_dir(self,work_dir):
		if self.out_dir:
			return self.out_dir
		else:
			self.out_dir = ensure_dir(os.path.join(work_dir,'%02d_%s'%(self.cnt,self.name)))
			return self.out_dir

class cPolyAseq:
	def __init__(self, args):

		self.fastq_dir = get_abspath_or_error(args.fastq_dir)
		self.work_dir = ensure_dir(args.work_dir)
		self.comp_group_fn = check_if_file_valid(args.comp_group_fn,error_flag=True)
		self.ctrl_tag = args.ctrl_tag
		self.exp_tag = args.exp_tag
		self.barcode_fn = args.barcode_fn
		self.fq_index_fn = check_if_file_valid(
			os.path.join(os.path.dirname(self.comp_group_fn), 'fastq_index.csv'),error_flag=True)

		self.bam_dir = None
		self.gfa_bam_dir = None
		self.analy_dir = None
		self.prepropa_rd = None

		self.pconf = self.load_config(args)
		self.step = 0 #categorical step
		self.sstep = 0 #serial unique step
		self.mSteps = []

	def get_log_dir(self):
		log_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'log')
		if not os.path.exists(log_dir):
			os.makedirs(log_dir)
		return log_dir

	def print_msg(self,mS,sstep_name):
		msg = "[%02d-%s][%02d-%s]"%(mS.cnt,mS.name,self.sstep,sstep_name)
		print(('%s in progress'%msg))

	def split_barcodes(self, run = True):
		self.step += 1 #1
		mS = mStep('ProcessSeq', self.step)
		self.mSteps.append(mS)
		sstep_name = 'split_barcodes'
		self.sstep += 1
		self.print_msg(mS,sstep_name)

		max_mismatch = 2
		out_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output', '%s_mm_%d' % (sstep_name,max_mismatch))
		ensure_dir(out_dir)

		log_prefix = os.path.join(self.get_log_dir(), '%02d_%s_mm'%(self.sstep,sstep_name))

		if (run and self.sstep >= self.pconf.resume):

			_ = split_barcodes_core(fastq_dir = self.fastq_dir,
															prog_path = self.pconf.sections[sstep_name]['bin_path'],
															out_dir = out_dir,
															log_prefix=log_prefix,
															barcode_fn=self.barcode_fn,
															max_mismatch = max_mismatch,
															ncpu = self.pconf.ncpu,
															debug=self.pconf.debug)

		self.split_barcodes_out_dir = out_dir

	def plot_barcodes(self,inpath):

		sstep_name = 'plot_barcodes'
		self.sstep += 1 #2
		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']
			log_prefix = os.path.join(self.get_log_dir(),'%02d_%s'%(self.sstep,sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s"%inpath,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def a_run_hist(self,demux_fastq_dir,mm=2):

		sstep_name = 'a_run_hist'
		self.sstep += 1 #3
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dirs = []
		prog_path = self.pconf.sections[sstep_name]['bin_path']

		outbase_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')
		ensure_dir(outbase_dir)
		suffix = '%02d_%s' % (self.sstep, sstep_name)

		log_prefix = os.path.join(self.get_log_dir(),suffix)
		for m in range(mm+1):
			suffix2 = "%s_mm%d"%(suffix,m)
			out_dir = os.path.join(outbase_dir,suffix2)
			ensure_dir(out_dir)

			cmd = "ls %s/*.fastq.gz | parallel -j %d %s --mm %d --outdir %s {}"%\
						(demux_fastq_dir,self.pconf.ncpu,prog_path,m,out_dir)

			if self.sstep >= self.pconf.resume:
				run_syscmd_wrapper(cmd,
													 stdout_fn="%s_%02d.out" % (log_prefix,m),
													 stderr_fn="%s_%02d.err" % (log_prefix,m),
													 debug = self.pconf.debug)

			out_dirs.append(out_dir)

		self.a_run_hist_out_dirs = out_dirs

	def a_run_hist_plot(self,inpath,demux_fastq_dir):
		sstep_name = 'a_run_hist_plot'
		self.sstep += 1 #4
		self.print_msg(self.mSteps[-1], sstep_name)
		outpath = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']
			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s'%(self.sstep,sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % inpath,
												 outopt="-o %s" % outpath,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def chop_a_tails(self,demux_fastq_dir):
		sstep_name = 'chop_a_tails'
		self.sstep += 1 #5
		prog_path = self.pconf.sections[sstep_name]['bin_path']
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output', sstep_name)

		if self.sstep >= self.pconf.resume:
			if not os.path.exists(out_dir):
				os.makedirs(out_dir)

			cmd = "ls %s/*.fastq.gz | parallel -j %d %s --outdir %s {}" % \
						(demux_fastq_dir, self.pconf.ncpu, prog_path, out_dir)

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(cmd,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		self.chop_a_tails_out_dir = out_dir

	def a_chop_plot(self,inpath):
		sstep_name = 'a_chop_plot'
		self.sstep += 1 #6
		self.print_msg(self.mSteps[-1], sstep_name)
		outpath = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']
			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s'%(self.sstep,sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % inpath,
												 outopt="-o %s" % outpath,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def make_bowtie_runs(self,inpath):
		sstep_name = 'make_bowtie_runs'
		self.sstep += 1 #7
		self.print_msg(self.mSteps[-1], sstep_name)
		self.bam_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'bowtie_alignments')
		ensure_dir(self.bam_dir)
		script_dir = os.path.join(self.bam_dir,'run_scripts')
		ensure_dir(script_dir)

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			runopt = "-f %s" % self.fq_index_fn

			runopt += " -n %d" % self.pconf.ncpu

			bowtie_index = self.pconf.sections['bowtie2-build']['ref_hg19_prefix']
			runopt += " -r %s" % bowtie_index

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % inpath,
												 outopt="-o %s" % script_dir,
												 runopt=runopt,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		script_fpaths = [os.path.join(script_dir,f) for f in os.listdir(script_dir) if f.endswith('.sh')]

		self.bowtie_scripts = script_fpaths

	def run_bowtie_ser(self,bowtie_scripts):
		sstep_name = 'run_bowtie_ser'
		self.sstep += 1 #8
		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:

			for c,bowtie_script in enumerate(bowtie_scripts):
				cmd = "bash %s" % bowtie_script

				log_prefix = os.path.join(self.get_log_dir(), '%02d_%s_%02d' % (self.sstep, sstep_name, c))

				run_syscmd_wrapper(cmd,
													 stdout_fn="%s.out" % log_prefix,
													 stderr_fn="%s.err" % log_prefix,
													 debug=self.pconf.debug)

	def make_filter_runs(self):
		self.step += 1
		mS = mStep('ProcessBams', self.step)
		self.mSteps.append(mS)

		sstep_name = 'make_filter_runs'
		self.sstep += 1 #9
		self.print_msg(self.mSteps[-1], sstep_name)
		mod_bam_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(),'%02d_%s' % (self.sstep, sstep_name))
			runopt = "-n %d"%self.pconf.ncpu
			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % self.bam_dir,
												 outopt="-o %s" % mod_bam_dir,
												 runopt=runopt,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

			sort_script_dir = os.path.join(mod_bam_dir, 'run_scripts_sort')
			filt_script_dir = os.path.join(mod_bam_dir, 'run_scripts_filter')

			so_script_fns = [os.path.join(sort_script_dir,f) for f in os.listdir(sort_script_dir) if f.endswith('.sh')]

			self.sort_script_fns = so_script_fns

			filt_script_fns = [os.path.join(filt_script_dir,f) for f in os.listdir(filt_script_dir) if f.endswith('.sh')]

			self.filt_script_fns = filt_script_fns

			self.mod_bam_dir = mod_bam_dir

			bam_index(mod_bam_dir)

			self.rm_pc_dups(mod_bam_dir)
		else:
			self.gfa_bam_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output', 'rm_int_a_7of10or6')
			self.sstep += 7

	def par_sort(self, scripts):
		sstep_name = 'par_sort'
		self.sstep += 1 #10

		if self.sstep >= self.pconf.resume:
			for c, script in enumerate(scripts):
				log_prefix = os.path.join(self.get_log_dir(), '%02d_%s_%02d' % (self.sstep, sstep_name, c))
				run_syscmd_wrapper("bash %s" % script,
													 stdout_fn="%s.out" % log_prefix,
													 stderr_fn="%s.err" % log_prefix,
													 debug=self.pconf.debug)

	def par_filter(self, scripts):
		sstep_name = 'par_filter'
		self.sstep += 1 #11

		if self.sstep >= self.pconf.resume:
			for c, script in enumerate(scripts):
				log_prefix = os.path.join(self.get_log_dir(), '%02d_%s_%02d' % (self.sstep, sstep_name, c))
				run_syscmd_wrapper("bash %s" % script,
													 stdout_fn="%s.out" % log_prefix,
													 stderr_fn="%s.err" % log_prefix,
													 debug=self.pconf.debug)

	def rm_pc_dups(self,mod_bam_dir):
		sstep_name = 'rm_pc_dups'
		self.sstep += 1 #12
		prog_path = self.pconf.sections[sstep_name]['bin_path']
		subd = '%02d_%s' % (self.sstep, sstep_name)
		dedup_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), subd)
		_ = ensure_dir(dedup_dir)
		if self.sstep >= self.pconf.resume:

			cmd = "ls %s/*.02_mapq*.bam | parallel -j %d %s --log --outdir %s {}" % \
						(mod_bam_dir, self.pconf.ncpu, prog_path, dedup_dir)

			log_prefix = os.path.join(self.get_log_dir(),subd)

			run_syscmd_wrapper(cmd,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

			bam_index(dedup_dir)

		self.dedup_dir = dedup_dir

	def plot_dups(self,dedup_dir):
		#work from here
		sstep_name = 'plot_dups'
		self.sstep += 1 #13

		out_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))
			runopt = "-f %s"%self.fq_index_fn
			run_syscmd_wrapper(prog_path,
												 inopt = "-i %s" % dedup_dir,
												 runopt = runopt,
												 outopt = "-o %s"% out_dir,
												 stdout_fn = "%s.out" % log_prefix,
												 stderr_fn = "%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def plot_alignment_stats(self,dedup_dir):
		sstep_name = 'plot_alignment_stats'
		self.sstep += 1 #14
		mod_bam_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))
			runopt = "-n %d"%self.pconf.ncpu
			run_syscmd_wrapper(prog_path,
												 inopt = "-i %s" % dedup_dir,
												 outopt = "-o %s" % mod_bam_dir,
												 runopt = runopt,
												 stdout_fn = "%s.out" % log_prefix,
												 stderr_fn = "%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def filter_genomic_a(self,dedup_dir):
		sstep_name = 'filter_genomic_a'
		self.sstep += 1 #15
		self.print_msg(self.mSteps[-1], sstep_name)
		self.gfa_bam_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output','rm_int_a_7of10or6')

		print((self.gfa_bam_dir))

		prog_path = self.pconf.sections[sstep_name]['bin_path']

		ref_genome_fa = self.pconf.sections['bowtie2-build']['ref_hg19_path']

		if self.sstep >= self.pconf.resume:
			if not os.path.exists(self.gfa_bam_dir):
				os.makedirs(self.gfa_bam_dir)

			cmd = "ls %s/*.bam | parallel -j%d %s --log --ref_genome_fa %s --outdir %s --win 10 --mm 3 --print --orfirst 6 {}" % \
						(dedup_dir, self.pconf.ncpu, prog_path, ref_genome_fa, self.gfa_bam_dir)

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(cmd,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

			bam_index(self.gfa_bam_dir)

	def plot_check_genomic_a(self):
		sstep_name = 'plot_check_genomic_a'
		self.sstep += 1 #16
		out_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')
		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % self.gfa_bam_dir,
												 outopt="-o %s" % out_dir,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def prepropa(self):
		self.step += 1 #17
		mS = mStep('CallApa', self.step)
		self.mSteps.append(mS)

		sstep_name = 'prepropa'
		self.sstep += 1
		self.print_msg(self.mSteps[-1], sstep_name)
		work_dir = self.mSteps[-1].get_out_dir(self.work_dir)
		self.prepropa_rd = os.path.join(work_dir, 'output', 'prepropa.rd')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			track_dir = os.path.join(work_dir, 'output', 'tracks')
			_ = ensure_dir(track_dir)

			runopt = "-t %s" % track_dir
			runopt += " -B %s" % self.gfa_bam_dir
			runopt += " -f %s" % self.pconf.sections['ftp_server_url']['value']
			runopt += " -b %s" % self.pconf.sections['ensembl']['biotype_table']
			runopt += " -g %s" % self.pconf.sections['ensembl']['gtf_file']
			runopt += " -T %s" % self.pconf.sections['cache_dir']['file_path']
			runopt += " -n %d" % self.pconf.ncpu

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % self.fq_index_fn,
												 outopt="-o %s" % self.prepropa_rd,
												 runopt=runopt,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def testbatchadj(self):

		sstep_name = 'testbatchadj'
		self.sstep += 1 #18
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir = self.mSteps[-1].get_out_dir(self.work_dir)
		out_fn = os.path.join(out_dir, 'output', 'apa.rd')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			runopt = "-n %d" % self.pconf.ncpu
			runopt += " -c %s" % self.comp_group_fn

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % self.prepropa_rd,
												 runopt=runopt,
												 outopt="-o %s" % out_fn,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		self.apa_rd =out_fn

	def alttests(self,apa_rd):
		sstep_name = 'alttests'
		self.sstep += 1 #19
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir = self.mSteps[-1].get_out_dir(self.work_dir)
		out_fn = os.path.join(out_dir, 'output', 'apa.alt.rd')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_rd,
												 runopt="-w %s" % self.prepropa_rd,
												 outopt="-o %s" % out_fn,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		self.apa_alt_rd = out_fn

	def callsig(self,apa_alt_rd):
		sstep_name = 'callsig'
		self.sstep += 1 #20
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir = self.mSteps[-1].get_out_dir(self.work_dir)

		out_fn = os.path.join(out_dir, 'output', 'apa.sig.rd')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_alt_rd,
												 runopt="-w %s" % self.prepropa_rd,
												 outopt="-o %s" % out_fn,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix)

		self.apa_sig_rd = out_fn

	def reporting(self,apa_sig_rd):
		sstep_name = 'reporting'
		self.sstep += 1 #21
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir= ensure_dir(os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output'))

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			runopt = "-w %s" % self.prepropa_rd
			runopt +=" -s %s" % self.pconf.sections['bowtie2-build']['chrom_size']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_sig_rd,
												 outopt="-o %s" % out_dir,
												 runopt=runopt,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)


	def intersect_gainedlost(self,apa_sig_rd):
		sstep_name = 'intersect_gainedlost'
		self.sstep += 1 #22
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir= ensure_dir(os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output'))

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			# runopt = "-w %s" % self.prepropa_rd
			runopt = " -c %s" % self.ctrl_tag
			runopt += " -e %s" % self.exp_tag

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_sig_rd,
												 outopt="-o %s" % out_dir,
												 runopt=runopt,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def plot_hexamer(self):
		sstep_name = 'plot_hexamer'
		self.sstep += 1 #23
		out_dir = os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output')

		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s -d %s" % (self.prepropa_rd,self.pconf.sections['polya_db2']['bed_path']),
												 outopt="-o %s" % out_dir,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

	def prepropa_to_excel(self):
		sstep_name = 'prepropa_to_excel'
		self.sstep += 1 #24
		in_dir = self.mSteps[-1].get_out_dir(self.work_dir)

		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % in_dir,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)


	def rep_cor(self):
		sstep_name = 'rep_cor'
		self.sstep += 1 #25
		in_dir = self.mSteps[-1].get_out_dir(self.work_dir)
		apa_sig_rd = os.path.join(in_dir, 'output', 'apa.sig.rd')

		self.print_msg(self.mSteps[-1], sstep_name)
		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i1 %s -i2 %s" % (self.prepropa_rd,apa_sig_rd),
												 outopt="-o %s" % os.path.join(in_dir, 'output'),
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)


	def mostupdown(self):

		prev_out_dir = self.mSteps[-1].get_out_dir(self.work_dir)
		apa_sig_rd = os.path.join(prev_out_dir, 'output', 'apa.sig.rd')

		self.step += 1 #26
		mS = mStep('AnnoApa', self.step)
		self.mSteps.append(mS)
		sstep_name = 'mostupdown'
		self.sstep += 1
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir= ensure_dir(os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output'))
		out_fn = os.path.join(out_dir, 'apa.ann.rd')

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_sig_rd,
												 outopt="-o %s" % out_fn,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		self.apa_sig_rd = apa_sig_rd

	def mostupdown_all_samples(self,apa_sig_rd):

		self.step += 1 #27
		mS = mStep('AnnoApa', self.step)
		self.mSteps.append(mS)
		sstep_name = 'mostupdown_all_samples'
		self.sstep += 1
		self.print_msg(self.mSteps[-1], sstep_name)
		out_dir= ensure_dir(os.path.join(self.mSteps[-1].get_out_dir(self.work_dir), 'output'))

		if self.sstep >= self.pconf.resume:
			prog_path = self.pconf.sections[sstep_name]['bin_path']

			log_prefix = os.path.join(self.get_log_dir(), '%02d_%s' % (self.sstep, sstep_name))

			run_syscmd_wrapper(prog_path,
												 inopt="-i %s" % apa_sig_rd,
												 outopt="-o %s" % out_dir,
												 stdout_fn="%s.out" % log_prefix,
												 stderr_fn="%s.err" % log_prefix,
												 debug=self.pconf.debug)

		return out_dir

	def load_config(self,args):
		"""
		objective: to load a program config file that contains subprogram paths, tools, run options, and databases
		:return:
		"""
		cConf = ProgramConfig(config_file=args.config_file,
													nnode=args.nnode,
													ncpu=args.ncpu,
													resume=args.resume,
													debug=args.debug)
		cConf.load_items()

		return cConf

parser = argparse.ArgumentParser(description="%s_v%s [author:%s]" % (PIPELINE_NAME,VERSION, author_email))
parser.add_argument('-C', '--config_file', dest='config_file', action='store', required=False, default=None,
                    help='a program configuration file [configs/program.conf]')
parser.add_argument('-i', '--fastq_dir', dest='fastq_dir', action='store', required=True, help='fastq directory.')

parser.add_argument('-b', '--barcode_fn', dest='barcode_fn', action='store', required=True, help='barcode_fn')

parser.add_argument('-e', '--exp_tag', dest='exp_tag', action='store', required=True, help='experimental group tag')
parser.add_argument('-c', '--ctrl_tag', action='store', required=True, help='control group tag')
parser.add_argument('-d', '--comp_group_fn', action='store', required=True, help='comp_group_fn')

parser.add_argument('-n', '--nnode', action='store', required=False, default=1, type=int,
                    help='the number of nodes to use in MPI mode [1];under development')
parser.add_argument('-p', '--ncpu', action='store', required=False, default=4, type=int,
                    help='the number of cpus to utilize [4]')
parser.add_argument('-o', '--work_dir', action='store', required=True, help='output directory without white space.')
parser.add_argument('-r', '--resume', action='store', required=False, default=-1, type=int,
                    help='specify the step number you want to resume if the successful result is available [-1]')
parser.add_argument('-s', '--sample_sheet', action='store', required=False, default=None, help='Sample sheet [None];this feature is under development.')
parser.add_argument('--debug', action='store_const', required=False, default=False, const=True, help='print commandline only in debug mode [False]')

args = parser.parse_args([
  '-C', '/content/apa_atingLab2019/01_polyAseq/configs/program.conf',
  '-i', '/content/apa_atingLab2019/01_polyAseq/01_wkd/fastq',
  '-e', 'DKO',
  '-c', 'HCT',
  '-d', '/content/apa_atingLab2019/01_polyAseq/01_wkd/comp_group.csv',
  '-o', '/content/apa_atingLab2019/01_polyAseq/01_wkd/out',
  '-b', '/content/apa_atingLab2019/01_polyAseq/01_wkd/barcode_list.csv',
  '-p', '6',
  '-r', '0'
])

cP = cPolyAseq(args)

In [ ]:
'''import paramiko
from scp import SCPClient

# 服务器信息
hostname = '74.121.151.55'
port = 22
username = 'root'
password = 'sewynLsefRPe'  # 替换为你的服务器密码

# 远程文件路径
remote_path = '/root/projects/apa_atingLab2019/01_polyAseq/01_wkd/out/01_ProcessSeq/output/chop_a_tails'
# 本地保存路径（Colab 环境）
local_path = '/content/apa_atingLab2019/01_polyAseq/01_wkd/out/01_ProcessSeq/output'
# 创建 SSH 客户端
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname, port, username, password)

# 使用 SCP 复制文件
with SCPClient(ssh.get_transport()) as scp:
    scp.get(remote_path, local_path, recursive=True)  # recursive=True 用于复制文件夹
'''

In [ ]:
# 拆分 launch_job 方法中的每一行代码，并打印输入参数
import copy
import pickle
import os

def save_state(obj, step_name):
    """保存对象的状态到文件"""
    state_dir = "state_files"
    os.makedirs(state_dir, exist_ok=True)
    state_file = os.path.join(state_dir, f"{step_name}.pkl")
    with open(state_file, "wb") as f:
        pickle.dump(obj, f)
    print(f"Saved state for step: {step_name} to {state_file}")

def load_state(step_name):
    """从文件加载对象的状态"""
    state_file = os.path.join("state_files", f"{step_name}.pkl")
    if os.path.exists(state_file):
        with open(state_file, "rb") as f:
            obj = pickle.load(f)
        print(f"Loaded state for step: {step_name} from {state_file}")
        return obj
    else:
        print(f"No state file found for step: {step_name}")
        return None


In [ ]:


"""逐步运行管道任务并保存状态"""
# Step 1: split_barcodes
cP01 = copy.deepcopy(cP)
print("Calling split_barcodes()")
cP01.split_barcodes(run=True)
save_state(cP01, "split_barcodes")

# Step 2: plot_barcodes
cP02 = load_state("split_barcodes") or cP01
print(f"Calling plot_barcodes({cP02.split_barcodes_out_dir})")
cP02.plot_barcodes(cP02.split_barcodes_out_dir)
save_state(cP02, "plot_barcodes")

# Step 3: a_run_hist
cP03 = load_state("plot_barcodes") or cP02
print(f"Calling a_run_hist({cP03.split_barcodes_out_dir}, mm=2)")
cP03.a_run_hist(cP03.split_barcodes_out_dir, mm=2)
save_state(cP03, "a_run_hist")

# Step 4: a_run_hist_plot
cP04 = load_state("a_run_hist") or cP03
print(f"Calling a_run_hist_plot({cP04.a_run_hist_out_dirs[1]}, {cP04.split_barcodes_out_dir})")
cP04.a_run_hist_plot(cP04.a_run_hist_out_dirs[1], cP04.split_barcodes_out_dir)
save_state(cP04, "a_run_hist_plot")

# Step 5: chop_a_tails
cP05 = load_state("a_run_hist_plot") or cP04
print(f"Calling chop_a_tails({cP05.split_barcodes_out_dir})")
cP05.chop_a_tails(cP05.split_barcodes_out_dir)
save_state(cP05, "chop_a_tails")

# Step 6: a_chop_plot
cP06 = load_state("chop_a_tails") or cP05
print(f"Calling a_chop_plot({cP06.chop_a_tails_out_dir})")
cP06.a_chop_plot(cP06.chop_a_tails_out_dir)
save_state(cP06, "a_chop_plot")

# Step 7: make_bowtie_runs
cP07 = load_state("a_chop_plot") or cP06
print(f"Calling make_bowtie_runs({cP07.chop_a_tails_out_dir})")
cP07.make_bowtie_runs(cP07.chop_a_tails_out_dir)
save_state(cP07, "make_bowtie_runs")

# Step 8: run_bowtie_ser
cP08 = load_state("make_bowtie_runs") or cP07
print(f"Calling run_bowtie_ser({cP08.bowtie_scripts})")
cP08.run_bowtie_ser(cP08.bowtie_scripts)
save_state(cP08, "run_bowtie_ser")

# Step 9: make_filter_runs
cP09 = load_state("run_bowtie_ser") or cP08
print("Calling make_filter_runs()")
cP09.make_filter_runs()
save_state(cP09, "make_filter_runs")

# Step 10: par_sort
cP10 = load_state("make_filter_runs") or cP09
print(f"Calling par_sort({cP10.sort_script_fns})")
cP10.par_sort(cP10.sort_script_fns)
save_state(cP10, "par_sort")

# Step 11: par_filter
cP11 = load_state("par_sort") or cP10
print(f"Calling par_filter({cP11.filt_script_fns})")
cP11.par_filter(cP11.filt_script_fns)
save_state(cP11, "par_filter")

# Step 12: bam_index
cP12 = load_state("par_filter") or cP11
print(f"Calling bam_index({cP12.mod_bam_dir})")
bam_index(cP12.mod_bam_dir)
save_state(cP12, "bam_index")

# Step 13: rm_pc_dups
cP13 = load_state("bam_index") or cP12
print(f"Calling rm_pc_dups({cP13.mod_bam_dir})")
cP13.rm_pc_dups(cP13.mod_bam_dir)
save_state(cP13, "rm_pc_dups")

# Step 14: plot_dups
cP14 = load_state("rm_pc_dups") or cP13
print(f"Calling plot_dups({cP14.dedup_dir})")
cP14.plot_dups(cP14.dedup_dir)
save_state(cP14, "plot_dups")

# Step 15: plot_alignment_stats
cP15 = load_state("plot_dups") or cP14
print(f"Calling plot_alignment_stats({cP15.dedup_dir})")
cP15.plot_alignment_stats(cP15.dedup_dir)
save_state(cP15, "plot_alignment_stats")

# Step 16: filter_genomic_a
cP16 = load_state("plot_alignment_stats") or cP15
print(f"Calling filter_genomic_a({cP16.dedup_dir})")
cP16.filter_genomic_a(cP16.dedup_dir)
save_state(cP16, "filter_genomic_a")

# Step 17: plot_check_genomic_a
cP17 = load_state("filter_genomic_a") or cP16
print("Calling plot_check_genomic_a()")
cP17.plot_check_genomic_a()
save_state(cP17, "plot_check_genomic_a")

# Step 18: prepropa
cP18 = load_state("plot_check_genomic_a") or cP17
print("Calling prepropa()")
cP18.prepropa()
save_state(cP18, "prepropa")

# Step 19: testbatchadj
cP19 = load_state("prepropa") or cP18
print("Calling testbatchadj()")
cP19.testbatchadj()
save_state(cP19, "testbatchadj")

# Step 20: alttests
cP20 = load_state("testbatchadj") or cP19
print(f"Calling alttests({cP20.apa_rd})")
cP20.alttests(cP20.apa_rd)
save_state(cP20, "alttests")

# Step 21: callsig
cP21 = load_state("alttests") or cP20
print(f"Calling callsig({cP21.apa_alt_rd})")
cP21.callsig(cP21.apa_alt_rd)
save_state(cP21, "callsig")

# Step 22: reporting
cP22 = load_state("callsig") or cP21
print(f"Calling reporting({cP22.apa_sig_rd})")
cP22.reporting(cP22.apa_sig_rd)
save_state(cP22, "reporting")

# Step 23: intersect_gainedlost
cP23 = load_state("reporting") or cP22
print(f"Calling intersect_gainedlost({cP23.apa_sig_rd})")
cP23.intersect_gainedlost(cP23.apa_sig_rd)
save_state(cP23, "intersect_gainedlost")

# Step 24: plot_hexamer
cP24 = load_state("intersect_gainedlost") or cP23
print("Calling plot_hexamer()")
cP24.plot_hexamer()
save_state(cP24, "plot_hexamer")

# Step 25: prepropa_to_excel
cP25 = load_state("plot_hexamer") or cP24
print("Calling prepropa_to_excel()")
cP25.prepropa_to_excel()
save_state(cP25, "prepropa_to_excel")

# Step 26: rep_cor
cP26 = load_state("prepropa_to_excel") or cP25
print("Calling rep_cor()")
cP26.rep_cor()
save_state(cP26, "rep_cor")

# Step 27: mostupdown
cP27 = load_state("rep_cor") or cP26
print("Calling mostupdown()")
cP27.mostupdown()
save_state(cP27, "mostupdown")

# Step 28: mostupdown_all_samples
cP28 = load_state("mostupdown") or cP27
print(f"Calling mostupdown_all_samples({cP28.apa_sig_rd})")
cP28.mostupdown_all_samples(cP28.apa_sig_rd)
save_state(cP28, "mostupdown_all_samples")

print("Pipeline completed successfully!")

